In [1]:
import os

In [2]:
%pwd

'd:\\Full Stack Data Science\\Time Series Analysis\\MAJOR PROJECT\\SMDF\\research'

In [3]:
cd ..

d:\Full Stack Data Science\Time Series Analysis\MAJOR PROJECT\SMDF


In [4]:
%pwd

'd:\\Full Stack Data Science\\Time Series Analysis\\MAJOR PROJECT\\SMDF'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [7]:
from SMDF.constants import *
from SMDF.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [10]:
import os
from SMDF.logging import logger
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [34]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    def get_data_transform(self):
        data = pd.read_csv(self.config.data_path)
        missing = data.isnull().sum()
        logger.info(missing)
        num_column =["M01AB","M01AE","N02BA","N02BE","N05B","N05C","R03","R06"]
        data[num_column].fillna("median",inplace=True)
        logger.info("filling Missing value done sucessfully !")
        data.set_index("datum",inplace=True)
        dict_lower = {}
        dict_upper = {}
        targets = ["M01AB","M01AE","N02BA","N02BE","N05B","N05C","R03","R06"]
        for i,var in enumerate(targets):
            irq = data[var].quantile(0.75) - data[var].quantile(0.25)
            lower_bridge = data[var].quantile(0.25) -(irq*1.5)
            upper_bridge = data[var].quantile(0.75) + (irq*1.5)
            print(f"Lower bound of {targets[i]}:{lower_bridge}")
            print(f"Upper bound of {targets[i]}:{upper_bridge}")
            print("**"*20)
            dict_lower[targets[i]] = [lower_bridge]
            dict_upper[targets[i]] = [upper_bridge]
        
        logger.info(f"suceesfully find upper and Lower bound")
        outlier_clmn = [] 
        for i in targets:
            x = [int(x) for x in dict_lower[i]]
            y = [int(y) for y in dict_upper[i]]
            
            if (x[0] and y[0]) > data[i].min():
                outlier_clmn.append(i)
        for i in outlier_clmn:
            x = [int(x) for x in dict_upper[i]]
            data.loc[data[i] >= x[0],i] = int(x[0])

        return data.to_csv("artifacts/data_transformation/salesDaily.csv",index=True)
        
        
    
    


In [35]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_data_transform()
except Exception as e:
    raise e

[2023-06-28 10:17:28,806: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-28 10:17:28,811: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-28 10:17:28,816: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-06-28 10:17:28,820: INFO: common: created directory at: artifacts]
[2023-06-28 10:17:28,821: INFO: common: created directory at: artifacts/data_transformation]
[2023-06-28 10:17:28,837: INFO: 337311858: Unnamed: 0    0
datum         0
M01AB         0
M01AE         0
N02BA         0
N02BE         0
N05B          0
N05C          0
R03           0
R06           0
dtype: int64]
[2023-06-28 10:17:28,846: INFO: 337311858: filling Missing value done sucessfully !]
Lower bound of M01AB:-2.505
Upper bound of M01AB:12.175
****************************************
Lower bound of M01AE:-1.8570000000000002
Upper bound of M01AE:9.335
****************************************
Lower bound of N02BA:-2.8000000000000007
Upper bound of N02BA:10.0


C:\Users\acer\AppData\Local\Temp\ipykernel_12692\337311858.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[num_column].fillna("median",inplace=True)
